In [1]:
import os
from keras.models import load_model
import pickle
import re
import numpy as np
import nltk
from nltk.corpus import stopwords

Using TensorFlow backend.
C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [2]:
def inNB():
    try:
        __IPYTHON__
        return True
    except NameError:
        return False

In [3]:
window_size=None
vocab_size=None
model=lambda:None
dictionary=dict()
stopWords=[]

In [4]:
def beforeStartup():
    if(not inNB()):
        maybe_download("binaries-v2.zip",
                    "https://blanksortbinaries.blob.core.windows.net/binaries/binaries-v2.zip")
        extract("binaries-v2.zip")
    globals()["model"] = load_model(os.path.join(os.path.join(os.path.abspath(
        ('.' if inNB() else '')+'./binaries'), "models"), "classification_model.h5"))
    model._make_predict_function()
    model.summary()
    names = ["window_size", "vocab_size", "dictionary"]
    for name in names:
        with open(os.path.join(os.path.join(os.path.abspath(('.' if inNB() else '')+'./binaries'), "pickles"), name+".pickle"), "rb") as f:
            globals()[name] = pickle.load(f)
    nltk.download("stopwords")
    globals()["stopWords"] = set(stopwords.words("english"))

In [5]:
def removeStopWords(rankedWords):
    return [word for word in rankedWords if word[0] not in stopWords]

In [6]:
def removeNumbers(rankedWords):
    return  [word for word in rankedWords if not any(char.isdigit() for char in word[0])]

In [7]:
def generateInverseCounts(wordSequence):
    countDict=dict()
    for word in wordSequence:
        if word not in countDict:
            countDict[word]=1
        else:
            countDict[word]+=1
    return [1/countDict[word] for word in wordSequence]

In [8]:
def findWordScores(wordSequence,tokenized):
    wordScores=np.zeros(len(wordSequence))
    contextCounts=np.zeros(len(wordSequence))
    inverseCounts=generateInverseCounts(wordSequence)
    for i in range(len(wordSequence)):
        leftBound=max(0,i-window_size)
        rightBound=min(len(wordSequence)-1,i+window_size)
        contextCounts[i]=rightBound-leftBound+1
        for k in range(i+1,rightBound+1):
            confidence=model.predict([[wordSequence[i]],[wordSequence[k]]])
            wordScores[i]+=confidence[0][0]
            wordScores[k]+=confidence[0][0]
    return ([tokenized[i],inverseCounts[i]*wordScores[i]/contextCounts[i]] for i in range(len(wordSequence)))

In [9]:
def formatData(string):
    string=re.sub(r"[^\w\s]", ' ', string)
    string=string.lower()
    tokenized=string.split()
    wordSequence=[]
    for word in tokenized:
        if word in dictionary:
            wordSequence.append(dictionary[word])
        else:
            wordSequence.append(dictionary["UNK"])
    return wordSequence,tokenized,string

In [10]:
def generateRankedList(string,**attributes):
    string=" ".join(string.split())
    noNumbers=attributes["noNumbers"] if "noNumbers" in attributes else True
    noStopWords=attributes["noStopWords"] if "noStopWords" in attributes else True
    wordSequence,tokenized,string=formatData(string)
    wordScores=findWordScores(wordSequence,tokenized)
    wordBank={}
    for word in wordScores:
        if word[0] in wordBank:
            wordBank[word[0]]=min(wordBank[word[0]],word[1])
        else:
            wordBank[word[0]]=word[1]
    sortedList=sorted(wordBank.items(), key=lambda x: x[1])
    rankedWords=[[x,y] for x,y in sortedList]
    if noStopWords:
        rankedWords=removeStopWords(rankedWords)
    if noNumbers:
        rankedWords=removeNumbers(rankedWords)
    return rankedWords

In [11]:
def printList(testCase,num=None,**attributes):
    noNumbers=attributes["noNumbers"] if "noNumbers" in attributes else True
    noStopWords=attributes["noStopWords"] if "noStopWords" in attributes else True
    rankedWords=generateRankedList(testCase,noNumbers=noNumbers,noStopWords=noStopWords)
    print(testCase)
    if(num==None):
        num=len(rankedWords)
    else:
        num=min(len(rankedWords),num)
    print()
    for i in range(num):
        print(str(i+1)+". "+rankedWords[i][0]+": "+str(round(rankedWords[i][1],3)))
    print()

In [12]:
if inNB():
    beforeStartup()

W1129 14:36:02.517243 25928 deprecation_wrapper.py:119] From C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1129 14:36:02.535192 25928 deprecation_wrapper.py:119] From C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1129 14:36:02.637917 25928 deprecation_wrapper.py:119] From C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1129 14:36:02.637917 25928 deprecation_wrapper.py:119] From C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecate